In [17]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv

In [18]:
load_dotenv()

True

In [19]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [20]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str
    feedback: int

In [21]:
def create_outline(state: BlogState) -> BlogState:

    title = state['title']

    prompt = f"Generate a detailed outline for the blog on the topic {title}"
    outline = model.invoke(prompt).content

    state['outline'] = outline
    return state



In [22]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f"Write a detailed blog on the title '{title}' with the following outline:\n{outline}\n\nMake sure to include relevant examples and explanations."
    content = model.invoke(prompt).content

    state['content'] = content
    return state

In [23]:
def rate_content(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']
    content = state['content']

    prompt = f"Rate the blog content on a scale of integer 1 to 10 based on the title '{title}' and the outline:\n{outline}\n\nContent:\n{content}\n\nProvide a score and brief feedback."
    feedback = model.invoke(prompt).content

    state['feedback'] = feedback
    return state

In [14]:
graph = StateGraph(BlogState)

#Node to generate a blog title
graph.add_node("create_outline",create_outline)
graph.add_node("create_blog", create_blog)
graph.add_node("rate_content", rate_content)

#edges
graph.add_edge(START, "create_outline")
graph.add_edge("create_outline", "create_blog")
graph.add_edge("create_blog", "rate_content")
graph.add_edge("rate_content", END)

#Compile
workflow = graph.compile()

In [24]:
initial_state = BlogState(title="The Future of AI in Everyday Life")

final_state = workflow.invoke(initial_state)
print(final_state['content'])

## The Future of AI in Everyday Life

**I. Introduction**

We live in a world increasingly shaped by unseen forces – algorithms that curate our newsfeeds, virtual assistants that anticipate our needs, and recommendation engines that guide our purchasing decisions.  These are all manifestations of Artificial Intelligence (AI), a rapidly evolving field that promises to revolutionize how we live, work, and interact with the world.  But what exactly *is* AI?  In simple terms, AI refers to computer systems capable of performing tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. This blog post will explore the growing integration of AI into our everyday lives, examining both its transformative potential and the challenges it presents. We'll delve into its impact on our homes, workplaces, healthcare systems, and transportation networks, ultimately considering the ethical and societal implications of this powerful technology.


**II. AI in 

In [27]:
print(rate_content(final_state)['feedback'])

I'd rate this blog content a **9 out of 10**.

**Feedback:**

The blog post is well-structured, following the outline effectively and covering all the key areas.  The writing is clear, concise, and engaging, avoiding overly technical jargon while still conveying complex information effectively.  The introduction effectively hooks the reader, and the conclusion provides a strong summary and call to action.  Each section logically flows into the next, creating a cohesive narrative.  The discussion of ethical implications is particularly strong, highlighting crucial concerns.

The only reason it doesn't receive a perfect 10 is the lack of the optional additions.  While the outline mentions images, videos, FAQs, and further reading, these are absent from the provided content.  Including these elements would significantly enhance reader engagement and the overall value of the blog post.  The addition of  visuals (even simple ones) would break up the text and make it more appealing.  FAQs wo